실습 목표
------------
* openAPI를 활용해서 데이터를 가져온다.
* xml데이터를 활용해서 필요한 정보를 뽑아내는 연습
* 필요한 데이터만 모두 가져오기

In [53]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pymysql

In [117]:
# Maria DB와 연동을 위해서 connect를 해준다.
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db='queant', charset='utf8mb4')
cur = conn.cursor()

In [55]:
#open api 주소를 url에 저장
url = "https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=47c0e868fdb16333d47d0e385641c3c0&topFinGrpNo=020000&pageNo=1"
data = urllib.request.urlopen(url).read().decode('euc-kr')

In [56]:
#string형태의 데이터를 xml객체로 만들어주기
root = ET.fromstring(data)
print(type(data), type(root))
#print(root)
#tree = ET.parse(root)

<class 'str'> <class 'xml.etree.ElementTree.Element'>


In [ ]:
#xml로 변환된 tree의 내용물 확인
ET.dump(root)

In [ ]:
# 각 상품들의 은행명만 뽑아와보기
for products in root.iter('kor_co_nm'):
    print(products.text)


In [21]:
#products -> product -> baseinfo -> 내용들
root[5][2][0][3].text


#상품 하나의 데이터를 뽑아와보기 (상품 하나의 내용들)
for child in root[5][0][0]:
    print(child.tag, child.text)

dcls_month 202206
fin_co_no 0010001
kor_co_nm 우리은행
fin_prdt_cd WR0001A
fin_prdt_nm 우리 SUPER정기예금
join_way 영업점,인터넷,스마트폰
mtrt_int 만기 후
- 1개월이내 : 만기시점약정이율×50%
- 1개월초과 6개월이내: 만기시점약정이율×30%
- 6개월초과 : 만기시점약정이율×20%

※ 만기시점 약정이율 : 일반정기예금 금리
spcl_cnd 해당사항 없음
join_deny 1
join_member 실명의 개인
etc_note -만기일을 일,월 단위로 자유롭게 선택 가능
-10년까지 자동재예치가 가능
max_limit None
dcls_strt_day 20220620
dcls_end_day None
fin_co_subm_day 202207141305


In [ ]:
#for 문을 이용해서 상품의 정보들을 뽑아와본다.
for product_tag in root[5]:
    print()
    for baseinfo_tag in product_tag[0]:
        print(baseinfo_tag.tag, baseinfo_tag.text)

In [ ]:
#for문을 활용해서 상품의 은행명, 상품명을 뽑아와본다.
for product_tag in root[5]:
    print(product_tag[0].find("kor_co_nm").text)
    print(product_tag[0].find("fin_prdt_nm").text)
    print()

In [114]:
# db와 python파일이 연결되었는지 확인.
cur.execute("SELECT * FROM queant.datas_article")

# 데이터를 읽어본다.
while True:
    row = cur.fetchone()
    if row == None:
        break
    data1 = row[0]
    data2 = row[1]
    data3 = row[2]
    print(data1, data2, data3)

204 하나은행 적금
205 우리은행 우리 SUPER정기예금
206 우리은행 시니어플러스우리예금 (회전형)
207 우리은행 WON 예금
208 우리은행 두루두루정기예금
209 한국스탠다드차타드은행 e-그린세이브예금
210 대구은행 DGB주거래우대예금(첫만남고객형)
211 대구은행 DGB행복파트너예금(일반형)
212 대구은행 DGB함께예금
213 대구은행 IM스마트예금
214 대구은행 친환경녹색예금
215 부산은행 백세청춘 실버예금
216 부산은행 저탄소 실천 예금
217 부산은행 내맘대로  예금
218 광주은행 미즈월복리정기예금
219 광주은행 호랏차차디지털예금
220 광주은행 KIA타이거즈우승기원예금
221 제주은행 제주Dream 정기예금 (개인/만기 지급식)
222 제주은행 사이버우대 정기예금 (만기지급식 -일반)
223 전북은행 JB시니어우대예금 (만기일시지급식)
224 전북은행 시장금리부정기예금 (만기일시지급식)
225 전북은행 스마트정기예금 (만기일시지급식)
226 전북은행 JB 다이렉트예금통장 (만기일시지급식)
227 전북은행 JB주거래예금
228 경남은행 BNK더조은정기예금
229 경남은행 BNK주거래우대정기예금
230 중소기업은행 IBK평생한가족통장(실세금리정기예금)
231 중소기업은행 i-ONE 놀이터예금
232 중소기업은행 1석7조통장(정기예금)
233 한국산업은행 정기예금
234 한국산업은행 KDBdream 정기예금
235 한국산업은행 KDB Hi 정기예금
236 국민은행 KB국민첫재테크예금
237 국민은행 KB골든라이프연금우대예금
238 국민은행 KB Young Youth 증여예금
239 국민은행 KB Star 정기예금
240 국민은행 KB Green Wave 1.5℃ 정기예금
241 국민은행 KB 더블모아 예금
242 신한은행 미래설계 크레바스 연금예금
243 신한은행 쏠편한 정기예금
244 신한은행 아름다운 용기 정기예금
245 신한은행 신한 S드림 정기예금
246 농협은행주식회사 NH왈츠회전예금 II
247 농협은행주식회사 e-금리우

In [118]:
# 쿼리문을 작성하고 우선 각 상품들의 은행명과 상품명을 DB에 저장해보기.
query = """INSERT INTO queant.datas_article (title, content) values (%s, %s);"""
for product_tag in root[5]:
    bank = product_tag[0].find("kor_co_nm").text
    prdt_name = product_tag[0].find("fin_prdt_nm").text
    prdt_name = prdt_name.replace("\n", " ")
    values = (bank, prdt_name) # 각각의 value들을 values에 묶어서 전달한다.
    cur.execute(query, values)
# 항상 데이터 조작을 했으면 커밋을 해야한다.
conn.commit()

In [119]:
# 제대로 저장되었는지 확인
cur.execute("SELECT * FROM queant.datas_article")

# 데이터를 읽어본다.
while True:
    row = cur.fetchone()
    if row == None:
        break
    data1 = row[0]
    data2 = row[1]
    data3 = row[2]
    print(data1, data2, data3)

204 하나은행 적금
205 우리은행 우리 SUPER정기예금
206 우리은행 시니어플러스우리예금 (회전형)
207 우리은행 WON 예금
208 우리은행 두루두루정기예금
209 한국스탠다드차타드은행 e-그린세이브예금
210 대구은행 DGB주거래우대예금(첫만남고객형)
211 대구은행 DGB행복파트너예금(일반형)
212 대구은행 DGB함께예금
213 대구은행 IM스마트예금
214 대구은행 친환경녹색예금
215 부산은행 백세청춘 실버예금
216 부산은행 저탄소 실천 예금
217 부산은행 내맘대로  예금
218 광주은행 미즈월복리정기예금
219 광주은행 호랏차차디지털예금
220 광주은행 KIA타이거즈우승기원예금
221 제주은행 제주Dream 정기예금 (개인/만기 지급식)
222 제주은행 사이버우대 정기예금 (만기지급식 -일반)
223 전북은행 JB시니어우대예금 (만기일시지급식)
224 전북은행 시장금리부정기예금 (만기일시지급식)
225 전북은행 스마트정기예금 (만기일시지급식)
226 전북은행 JB 다이렉트예금통장 (만기일시지급식)
227 전북은행 JB주거래예금
228 경남은행 BNK더조은정기예금
229 경남은행 BNK주거래우대정기예금
230 중소기업은행 IBK평생한가족통장(실세금리정기예금)
231 중소기업은행 i-ONE 놀이터예금
232 중소기업은행 1석7조통장(정기예금)
233 한국산업은행 정기예금
234 한국산업은행 KDBdream 정기예금
235 한국산업은행 KDB Hi 정기예금
236 국민은행 KB국민첫재테크예금
237 국민은행 KB골든라이프연금우대예금
238 국민은행 KB Young Youth 증여예금
239 국민은행 KB Star 정기예금
240 국민은행 KB Green Wave 1.5℃ 정기예금
241 국민은행 KB 더블모아 예금
242 신한은행 미래설계 크레바스 연금예금
243 신한은행 쏠편한 정기예금
244 신한은행 아름다운 용기 정기예금
245 신한은행 신한 S드림 정기예금
246 농협은행주식회사 NH왈츠회전예금 II
247 농협은행주식회사 e-금리우

In [120]:
# 마지막에는 연결을 끊어준다.
conn.close()